In [ ]:
using Pkg
Pkg.add(PackageSpec(url="https://github.com/artem-gordeev/ColdAtoms.jl"))

In [ ]:
fontsize = 14;
linewidth = 2;
color = "red";

In [ ]:
using QuantumOptics
include("../params/default.jl")

N = 10;
samples, acc_rate = samples_generate(trap_params, atom_params, N; skip=5000, freq=1000);

detuning_params = [Δ0, δ_twophoton(Ωr, Ωb, Δ0)];

n=4
blue_laser_params = [Ωb, wb, zb, "simp_flattop_LG", n, n];

T0 = T_twophoton(Ωr, Ωb, Δ0);
tspan = [0.0:T0/10:1*T0;];
ψ0 = g;

In [ ]:
ρ_mean, ρ2_mean = simulation_blue_intens(
    tspan, ψ0,  
    atom_params,    trap_params,     samples,
    red_laser_params,    blue_laser_params,
    detuning_params,    decay_params;
    atom_motion = true,    free_motion = true,   
    spontaneous_decay = true, parallel = false
    );
    
Pg = real(expect(g ⊗ dagger(g), ρ_mean)); 
Pr = real(expect(r ⊗ dagger(r), ρ_mean));  

In [ ]:
using PlotlyJS
trace1 = scatter(x=tspan, y=Pr,
mode="lines+markers",
name="modeling")
plot([trace1])   

In [ ]:
using LsqFit
@. model(x, p) = 0.5*(1 - exp(-p[2]*x)*cos(p[1]*x)) 
p0 = [18.0,1.0]
fit = curve_fit(model, tspan, Pr, p0)
print(coef(fit)[2]/coef(fit)[1])

In [ ]:
using PlotlyJS
trace1 = scatter(x=tspan, y=Pr,
mode="lines+markers",
name="modeling")
trace2 = scatter(x=tspan, y=model(tspan, coef(fit)),
mode="lines+markers",
name="decaying rabi fit")
plot([trace1,trace2])   

In [ ]:
ρ_mean, ρ2_mean = 
    simulation(
        tspan, ψ0, 
        
        atom_params,
        trap_params,
        samples,
        
        f,
        red_laser_phase_amplitudes,
        blue_laser_phase_amplitudes,
        
        red_laser_params,
        blue_laser_params,
        
        detuning_params,
        decay_params;
        parallel=false,
        free_motion=true
    );
    
Pg = real(expect(g ⊗ dagger(g), ρ_mean)); 
Pr = real(expect(r ⊗ dagger(r), ρ_mean));  